In [1]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import pandas as pd
from openpyxl import load_workbook
import re
import io
import os

# basic input ------------------------------------------------
 
def eq_scripts_load(_sep, _subdir, _file):
    
    # if specific file selected it should be XLSX one
    if _file != "":
        
        if _subdir != '':
            _subdir = '/' + _subdir
        _subdir = '../../input' + _subdir + '/'

        _file = _subdir + _file
        
        # open excel file
        with open(_file, "rb") as f:
            inmemory_file = io.BytesIO(f.read())
        wb = load_workbook(inmemory_file, read_only = True)
        
        # read data
        r = re.compile(r'^(input\_)*stoich(iometric)*\_coefficients*$')
        st_coeff_data = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0])
        
        r = re.compile(r'^(input\_)*concentrations*$')
        con_data = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0], header = 1)
        
        r = re.compile(r'^(input\_)*concentrations*$')
        type_con = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0]
                                 , header = None, nrows = 1).iloc[0,:]
        
        r = re.compile(r'^(input\_)*k\_constants\_log10$')
        lg_k_data = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0])
        
        r = re.compile(r'heats')
        heats_data = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0])
        
        r = re.compile(r'targets')
        targets = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0], header = None)
        
        r = re.compile(r'enthalpies')
        delta_H = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0])
        
        r = re.compile(r'^(particle|component)_names*$')
        component_name_for_yields = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0]
                                                  , header = None).iat[0, 0]
        
    # use a bunch of plain text files instead
    else:
          
        if _subdir != '':
            _subdir = '/' + _subdir
        _subdir = '../../input' + _subdir + '/'

        file_names = list(os.listdir(path = _subdir))

        r = re.compile(r'^(input\_)*stoich(iometric)*\_coefficients*')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        st_coeff_data = pd.read_csv(file, sep = _sep)

        r = re.compile(r'^(input\_)*k\_constants\_log10')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        lg_k_data = pd.read_csv(file, sep = _sep, decimal = ",")
        
        r = re.compile(r'^(input\_)*concentrations*')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        con_data = pd.read_csv(file, sep = _sep, decimal = ",", header = 1)
        
        type_con = pd.read_csv(file, sep = _sep, header = None, nrows = 1).iloc[0,:]
        
        r = re.compile(r'heats')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        heats_data = pd.read_csv(file, sep = _sep, decimal = ",")
        
        r = re.compile(r'targets')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        targets = pd.read_csv(file, sep = _sep, header = None)
        
        r = re.compile(r'enthalpies')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        delta_H = pd.read_csv(file, sep = _sep)
        
        r = re.compile(r'^(particle|component)_names*')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        component_name_for_yields = pd.read_csv(file, header = None).iat[0, 0]

    return st_coeff_data, lg_k_data, con_data, type_con, heats_data, targets, delta_H, component_name_for_yields  

In [2]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import numpy as np
import re

# basic preprocessing ----------------------------------------
    
def eq_preproc(st_coeff_data, lg_k_data, con_data, type_con, heats_data, targets, delta_H, component_name_for_yields):
    
    # checking if there are several series
    
    if 'series' not in con_data.columns:        
        con_data['series'], type_con[np.shape(st_coeff_data)[1]] = '', ''
    
    if 4 not in heats_data.index:
        heats_data.loc[4] = ''
        print(heats_data)

    # series variables
    
    ser_info = con_data['series'].to_numpy()
    ser_unique = np.unique(ser_info)
    ser_num = np.shape(np.unique(ser_info))[0]

    # matrix of stoich coeff with formal reactions added
    st_coeff_matrix = st_coeff_data.drop('name', axis = 1).to_numpy()
    formal_matrix = np.eye(np.shape(st_coeff_matrix)[1], dtype = int)
    st_coeff_matrix = np.vstack((formal_matrix, st_coeff_matrix))
        
    # product names lists : full and base components only
    
    prod_names_con = list(con_data.drop('series', axis = 1))
    prod_names = prod_names_con + st_coeff_data['name'].tolist()
    
    # creating the vector of equilibrium constants including the formal reactions
    lg_k = (np.vstack((np.zeros((np.shape(st_coeff_matrix)[1], 1)), lg_k_data.to_numpy().astype(float))))
    
    # checking the consistency of reagent names in different sheets    
    if prod_names_con != list(st_coeff_data.drop('name', axis = 1)):
        print('Check the consistency of reagent names!')
    
    # split concentrations matrix
    #con_matrix = [g for _, g in con_data.groupby(['series'])]
        
    #for cnm_index, cnm in enumerate(con_matrix):
        #con_matrix[cnm_index] = cnm.drop('series', axis = 1).to_numpy().astype(float)
    con_matrix = con_data.drop('series', axis = 1).to_numpy()
    ser_counts = con_data.groupby(['series']).size().tolist();
    
    # creating vector of indices of components with predetermined concentrations
    ign_indices = np.array(type_con.index[type_con == 'eq'])
    
    # reading volumes from experimental data
    volumes = heats_data.drop('data', axis = 1).to_numpy()[0]
    
    # reading exp heats from experimental data
    heats = heats_data.drop('data', axis = 1).to_numpy()[1] - heats_data.drop('data', axis = 1).to_numpy()[2]
        
    devs = heats_data.drop('data', axis = 1).to_numpy()[3]
    
    # creating vector of known enthalpies
    dH_par = np.hstack((np.zeros(len(prod_names_con)), np.transpose(delta_H.drop('Reaction', axis = 1).to_numpy())[0]))
    
    # number of constant to find
    tar_names = set(targets.to_numpy()[0][1:])
    tar_num = [index for index, item in enumerate(prod_names) if item in tar_names]
    
    # number of enthalpy to find
    dH_names = np.hstack((prod_names_con, np.transpose(delta_H.drop('Value', axis = 1).to_numpy())[0]))
    un_el = set(prod_names) - set(dH_names)
    dH_ind_wtf = list([index for index, item in enumerate(prod_names) if item in un_el])
    un_el = set(prod_names) - (set(prod_names) - set(dH_names))
    dH_ind = list([index for index, item in enumerate(prod_names) if item in un_el])
    
    if component_name_for_yields not in prod_names:
        print('The component name for partition should be among those of basis components')
        
    idx, = np.where(component_name_for_yields == np.array(prod_names_con))
    
    return ser_num, st_coeff_matrix, prod_names, lg_k, prod_names_con, con_matrix, ign_indices, ser_counts, ser_info, type_con, volumes, heats, devs, tar_num, dH_ind, dH_ind_wtf, dH_par, idx  
    # ser_num, ser_counts, ser_info not further used yet! 

In [3]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import math
import numpy as np
from copy import copy, deepcopy

def eq_calc(max_iter, eps, ser_num, st_coeff_matrix, type_con, lg_k,
            con_matrix, ign_indices, ser_counts, ser_info): # ser_num, ser_counts, ser_info not further used yet!
    
    c_res_out = [0] * len(con_matrix)
    
    for k in range(np.shape(con_matrix)[0]):
                
        reag_eq_con_matrix = deepcopy(con_matrix[k]) # initial estimation of equilibrium concentrations of reagents
        init_conc = deepcopy(con_matrix[k]) # value for residual calculation in inner function
        
        prod_eq_con_matrix = inner_eq_calc(reag_eq_con_matrix, max_iter, lg_k, st_coeff_matrix, init_conc, ign_indices, eps)   
                    
        c_res_out[k] += prod_eq_con_matrix[0]

    return c_res_out

def inner_eq_calc(reag_eq_con_matrix, max_iter, lg_k, st_coeff_matrix, init_conc, ign_indices, eps): 
    
    lg_k, st_coeff_matrix, con_matrix = deepcopy(lg_k), deepcopy(st_coeff_matrix), deepcopy(init_conc)
    
    # if some equilibrium concentrations are set
    if np.shape(ign_indices)[0] > 0:

        range_start = st_coeff_matrix.shape[1]
        range_end = lg_k.shape[0]
        
        lg_k_upd = np.matmul(st_coeff_matrix[range_start:range_end, ign_indices],
          np.log10(init_conc[ign_indices])) #+ lg_k[range_start:range_end]
        
        lg_k[range_start:range_end] = lg_k_upd.reshape(-1, 1) + lg_k[range_start:range_end]
        
        init_conc = np.delete(init_conc, ign_indices, axis = 0)
        reag_eq_con_matrix = np.delete(reag_eq_con_matrix, ign_indices, axis = 0)
        st_coeff_matrix = np.delete(st_coeff_matrix, ign_indices, axis = 1)

    # start of iterative procedure
    for it in range(max_iter):

        # caclulating the equilibrium concentrations of products
        prod_eq_con_matrix = np.exp(np.transpose(np.array(math.log(10) * np.array(lg_k))) +
                                    np.dot(st_coeff_matrix, np.log(reag_eq_con_matrix)))
            
        # calculating the total concentrations of reagents
        reag_tot_con_matrix_calc = np.transpose(np.dot(np.transpose(st_coeff_matrix), np.transpose(prod_eq_con_matrix)))
            
        # calculating the residuals
        g_res = np.array(reag_tot_con_matrix_calc) - np.array(init_conc)
                            
        # calculating the Jacobi matrices
        jac_matrix = np.dot(np.transpose(st_coeff_matrix), (np.array(st_coeff_matrix) * np.transpose(prod_eq_con_matrix)))   
        
        # new estimation of equilibrium concentrations of reagents
        prev = np.log(reag_eq_con_matrix)
        reag_eq_con_matrix = np.exp(prev - np.transpose(np.dot(np.linalg.inv(jac_matrix), np.transpose(g_res))))
        reag_eq_con_matrix = reag_eq_con_matrix[0]
        error = abs(np.log(reag_eq_con_matrix) - prev)
                
        # checking the convergence
        if np.max(error) < eps:
               
            # if some equilibrium concentrations are set
            if np.shape(ign_indices)[0] > 0:
                prod_eq_con_matrix[:, ign_indices] = con_matrix[ign_indices]
                        
            break

    return prod_eq_con_matrix   

In [4]:
# input for xlsx file
_subdir = "calorimetry"
_sep = ";"
_file = "test_1.xlsx"

max_iter, eps = 1000, 0.0000001

st_coeff_data, lg_k_data, con_data, type_con, heats_data, targets, delta_H, component_name_for_yields = eq_scripts_load(_sep, _subdir, _file)

(ser_num, st_coeff_matrix, prod_names, lg_k, prod_names_con, con_matrix, ign_indices, ser_counts,
 ser_info, type_con, volumes, heats, devs, tar_num, dH_ind, dH_ind_wtf, dH_par, idx) = eq_preproc(st_coeff_data,
                                                                                             lg_k_data, con_data, type_con,
                                                                                             heats_data, targets, delta_H, component_name_for_yields)

c_res_out = eq_calc(max_iter, eps, ser_num, st_coeff_matrix, type_con, lg_k,
            con_matrix, ign_indices, ser_counts, ser_info)

print('\nStoich coeff data')
print(st_coeff_data)

print('\nlg K')
print(lg_k_data)

print('\nconcentrations')
print(con_data)

print('\ntype con')
print(type_con)

print('\nexperimental data')
print(heats_data)

print('\ntargets')
print(targets)

print('\ndelta_H')
print(delta_H)

          data      1           2           3           4            5  \
0      volumes     15      15.025       15.05      15.075         15.1   
1  observation      0    0.069951    0.069513    0.069854     0.071474   
2     dilution      0 -0.00195354  0.00247103 -0.00017689  0.000556654   
3    deviation  1e-06       1e-06       1e-06       1e-06        1e-06   
4                                                                        

            6           7           8            9          10           11  \
0      15.125       15.15      15.175         15.2      15.225        15.25   
1    0.067108    0.066289    0.056026     0.044493    0.027882     0.009239   
2  0.00070773  0.00185424  0.00108107  0.000473122  0.00106313  0.000271799   
3       1e-06       1e-06       1e-06        1e-06       1e-06        1e-06   
4                                                                             

           12           13           14          15           16  
0      15.275

In [5]:
print('\nSeries')
print(ser_num)
print(ser_counts)
print(ser_info)

print('\nSt coeff matr')
print(st_coeff_matrix)

print('\nprod names')
print(prod_names)
print(prod_names_con)

print('\nconst')
print(lg_k)

print('\nconcentrations')
print(con_matrix)

print(ign_indices)

print('\ntype con')
print(type_con)

print('\nvolumes')
print(volumes)

print('\nheats')
print(heats)

print('\ndeviations')
print(devs)

print('\nwhat to find and what is known')
print(tar_num)
print(dH_ind)
print(dH_ind_wtf)

print('\nenthalpies')
print(dH_par)


Series
1
[16]
['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']

St coeff matr
[[1 0]
 [0 1]
 [1 1]]

prod names
['PLP', 'T3H', 'Comp']
['PLP', 'T3H']

const
[[0.  ]
 [0.  ]
 [5.13]]

concentrations
[[7.97022000e-04 1.00000000e-18]
 [7.95695999e-04 9.91228990e-05]
 [7.94374403e-04 1.97916525e-04]
 [7.93057190e-04 2.96382517e-04]
 [7.91744338e-04 3.94522502e-04]
 [7.90435825e-04 4.92338095e-04]
 [7.89131631e-04 5.89830903e-04]
 [7.87831733e-04 6.87002520e-04]
 [7.86536110e-04 7.83854532e-04]
 [7.85244742e-04 8.80388513e-04]
 [7.83957608e-04 9.76606026e-04]
 [7.82674686e-04 1.07250863e-03]
 [7.81395956e-04 1.16809785e-03]
 [7.80121398e-04 1.26337525e-03]
 [7.78850991e-04 1.35834233e-03]
 [7.77584715e-04 1.45300060e-03]]
[]

type con
0    tot
1    tot
3       
Name: 0, dtype: object

volumes
[15.0 15.024997 15.049994 15.074991 15.099988 15.124985 15.149982
 15.174979 15.199976 15.224973 15.24997 15.274967 15.299964 15.324961
 15.349958 15.374955]

heats
[0.0 0.071904538 0.067041971 0.070

In [6]:
print('\nEq conc')
print(np.array(c_res_out))
print(np.shape(c_res_out))


Eq conc
[[7.97022000e-04 9.21528957e-21 9.90784710e-19]
 [6.97615339e-04 1.04223909e-06 9.80806599e-05]
 [5.98877798e-04 2.41992000e-06 1.95496605e-04]
 [5.00996218e-04 4.32154539e-06 2.92060972e-04]
 [4.04324982e-04 7.10314595e-06 3.87419356e-04]
 [3.09610296e-04 1.15125657e-05 4.80825529e-04]
 [2.18643156e-04 1.93424280e-05 5.70488475e-04]
 [1.36273225e-04 3.54440128e-05 6.51558508e-04]
 [7.40297191e-05 7.13481412e-05 7.12506391e-04]
 [4.06485875e-05 1.35792358e-04 7.44596155e-04]
 [2.57377216e-05 2.18386140e-04 7.58219886e-04]
 [1.83824627e-05 3.08216402e-04 7.64292223e-04]
 [1.41869711e-05 4.00888869e-04 7.67208985e-04]
 [1.15159629e-05 4.94769811e-04 7.68605436e-04]
 [9.67796265e-06 5.89169297e-04 7.69173031e-04]
 [8.33995108e-06 6.83755840e-04 7.69244770e-04]]
(16, 3)


In [7]:
dif_conc, devi, d_heats, dV, con_matrix_red = [], [], [], [], [] 

# finding the increment in equilibrium concentrations of products - needed for dH calculation

for s in range(ser_num):
    
    for k in range(ser_counts[s] - 1):
        
        dif_conc.append(c_res_out[int(np.sum(ser_counts[:s])) + k + 1] - c_res_out[int(np.sum(ser_counts[:s])) + k])

        devi.append(devs[int(np.sum(ser_counts[:s])) + k + 1])
        
        d_heats.append(heats[int(np.sum(ser_counts[:s])) + k + 1])
        
        #dV.append(volumes[int(np.sum(ser_counts[:s])) + k + 1] - volumes[int(np.sum(ser_counts[:s])) + k])
        
        con_matrix_red.append(con_matrix[int(np.sum(ser_counts[:s])) + k + 1])

# transforming deviation to the matrix form
devs = np.diag((1 / (np.array(devi) * np.array(devi))) * (np.sum(np.array(devi) * np.array(devi)) / len(devi))) 
dV = volumes[1:] # which one of lists of volumes should be used - the differences or the total ones - 
                # depends on the type of calorimeter! Just freaking amazing!
                # Got an idea how to fix it later on. Gotta check if volumes in the initial data, 
                # if no, the vector containing 1 should be created instead

dif_conc, d_heats, dV, con_matrix_red = np.array(dif_conc), np.array(d_heats), np.array(dV).astype(float), np.array(con_matrix_red)

print(dV)
# finding vector of experimental values

q_corr = d_heats
for i in range(len(d_heats)):
    for j in range(len(dV[dH_ind])):
        q_corr[i] -= dV[dH_ind[j]]*dif_conc[i, dH_ind[j]]*dH_par[j]

A = dV[dH_ind_wtf] * dif_conc[:, dH_ind_wtf] #/ con_matrix_red[:, idx]
AT = np.transpose(A)
LTP = np.linalg.inv(np.dot(np.dot(AT, devs), A))
RTP = np.dot(np.dot(AT, devs), q_corr)
dH_res = np.dot(LTP, RTP)
print(dH_res)

[15.024997 15.049994 15.074991 15.099988 15.124985 15.149982 15.174979
 15.199976 15.224973 15.24997  15.274967 15.299964 15.324961 15.349958
 15.374955]
[47.57917279]
